# Text-to-SQL: Prompt Engineering

This notebook explores prompt optimization strategies for Text-to-SQL generation.

## Objectives
1. Test different prompt templates
2. Optimize few-shot examples
3. Evaluate schema formatting strategies
4. Analyze prompt token usage vs. accuracy trade-offs

In [ ]:
import sys
import os
import json
import pandas as pd
from pathlib import Path
from typing import List, Dict
import time

# Add src to path
sys.path.append(str(Path('../src').resolve()))

from schema_manager import SchemaManager
from query_generator import TextToSQLGenerator, QueryResult
from prompt_templates import PromptTemplates
from query_validator import QueryValidator

# Load environment variables
# Note: Set OPENAI_API_KEY environment variable or uncomment:
# os.environ['OPENAI_API_KEY'] = 'your-key-here'

## 1. Baseline Setup

In [ ]:
# Initialize components
db_path = "../data/sample_database.db"
schema_manager = SchemaManager(db_path)
validator = QueryValidator(db_path)

# Load test queries
with open('../data/test_queries.json', 'r') as f:
    test_queries = json.load(f)

print(f"Loaded {len(test_queries)} test queries")
print(f"Complexity distribution: {pd.Series([q['complexity'] for q in test_queries]).value_counts().to_dict()}")

## 2. Schema Formatting Experiments

In [ ]:
# Test different schema formats
schema_formats = ['create_table', 'compact', 'json']

print("Schema Format Comparison:")
print("=" * 80)

for format_type in schema_formats:
    schema_str = schema_manager.format_schema(
        format_type=format_type,
        include_sample_rows=False
    )
    token_estimate = len(schema_str) // 4  # Rough token estimate
    
    print(f"\nFormat: {format_type}")
    print(f"Characters: {len(schema_str)}")
    print(f"Estimated tokens: {token_estimate}")
    print("-" * 40)
    print(schema_str[:300] + "..." if len(schema_str) > 300 else schema_str)
    print("=" * 80)

In [ ]:
# Compare with sample rows included
print("\nSchema with Sample Rows (CREATE TABLE format):")
print("=" * 80)

schema_with_samples = schema_manager.format_schema(
    format_type='create_table',
    include_sample_rows=True,
    num_sample_rows=3
)

print(f"Characters: {len(schema_with_samples)}")
print(f"Estimated tokens: {len(schema_with_samples) // 4}")
print("\nSample:")
print(schema_with_samples[:500] + "...")

## 3. Few-Shot Example Selection

In [ ]:
# Analyze default few-shot examples
prompt_templates = PromptTemplates()
examples = prompt_templates.few_shot_examples

print("Default Few-Shot Examples:")
print("=" * 80)
for i, example in enumerate(examples, 1):
    print(f"\nExample {i}:")
    print(f"Question: {example['question']}")
    print(f"SQL: {example['sql']}")
    print(f"Explanation: {example['explanation']}")
    print("-" * 80)

print(f"\nTotal examples: {len(examples)}")
total_chars = sum(len(ex['question']) + len(ex['sql']) + len(ex['explanation']) for ex in examples)
print(f"Total characters: {total_chars}")
print(f"Estimated tokens: {total_chars // 4}")

In [ ]:
# Test dynamic example selection
test_question = "What are the top 5 products by revenue?"

# Build prompt with selective examples
schema_str = schema_manager.format_schema(format_type='create_table', include_sample_rows=False)
prompt = prompt_templates.build_text_to_sql_prompt(
    schema=schema_str,
    question=test_question,
    include_examples=True
)

print("Full Prompt:")
print("=" * 80)
print(prompt)
print("=" * 80)
print(f"\nPrompt length: {len(prompt)} characters")
print(f"Estimated tokens: {len(prompt) // 4}")

## 4. Prompt Template Variations

In [ ]:
# Test different system message styles
system_messages = {
    'default': prompt_templates.get_system_message(),
    'concise': """You are a SQL expert. Convert natural language to SQLite queries.
Rules:
- Return ONLY the SQL query
- Use proper SQLite syntax
- Be precise and efficient""",
    'detailed': """You are an expert database assistant specialized in converting natural language 
questions into accurate SQLite queries. Your task is to analyze the user's question, understand 
the database schema, and generate a syntactically correct and semantically meaningful SQL query.

Guidelines:
1. Carefully analyze the question to understand what data is being requested
2. Identify which tables and columns are needed
3. Use appropriate JOINs when multiple tables are involved
4. Apply filters (WHERE) to match the question's constraints
5. Use aggregations (COUNT, SUM, AVG) when the question asks for statistics
6. Sort results (ORDER BY) when rankings or orderings are requested
7. Limit results if the question asks for "top N" items
8. Handle NULL values appropriately
9. Return ONLY the SQL query without explanations"""
}

for name, msg in system_messages.items():
    print(f"\n{name.upper()} System Message:")
    print("=" * 80)
    print(msg)
    print(f"\nLength: {len(msg)} characters, ~{len(msg) // 4} tokens")
    print("=" * 80)

## 5. Prompt Optimization Experiments

Test different prompt configurations on sample queries.

In [ ]:
# Define prompt configurations to test
configs = [
    {
        'name': 'minimal',
        'schema_format': 'compact',
        'include_samples': False,
        'num_examples': 0,
    },
    {
        'name': 'standard',
        'schema_format': 'create_table',
        'include_samples': False,
        'num_examples': 3,
    },
    {
        'name': 'comprehensive',
        'schema_format': 'create_table',
        'include_samples': True,
        'num_examples': 5,
    }
]

# Calculate token usage for each config
test_question = "Show me all customers who have placed orders over $100"

print("Prompt Configuration Comparison:")
print("=" * 80)

for config in configs:
    schema_str = schema_manager.format_schema(
        format_type=config['schema_format'],
        include_sample_rows=config['include_samples']
    )
    
    # Build prompt
    if config['num_examples'] > 0:
        examples = prompt_templates.few_shot_examples[:config['num_examples']]
        prompt = prompt_templates.build_text_to_sql_prompt(
            schema=schema_str,
            question=test_question,
            include_examples=True
        )
    else:
        prompt = f"Schema:\n{schema_str}\n\nQuestion: {test_question}\n\nSQL:"
    
    token_estimate = len(prompt) // 4
    
    print(f"\nConfig: {config['name']}")
    print(f"  Schema format: {config['schema_format']}")
    print(f"  Include samples: {config['include_samples']}")
    print(f"  Num examples: {config['num_examples']}")
    print(f"  Estimated tokens: {token_estimate}")
    print("-" * 80)

## 6. Live Testing (Requires API Key)

In [ ]:
# Test with actual API calls (uncomment if API key is set)
"""
# Initialize generator
generator = TextToSQLGenerator(db_path, provider='openai', model_name='gpt-3.5-turbo')

# Select a few test queries
test_sample = [
    q for q in test_queries 
    if q['complexity'] in ['simple', 'medium']
][:5]

results = []
for query in test_sample:
    print(f"\nTesting: {query['question']}")
    print(f"Expected: {query['expected_sql']}")
    
    # Generate SQL
    result = generator.generate_sql(
        question=query['question'],
        return_reasoning=True
    )
    
    print(f"Generated: {result.sql}")
    print(f"Status: {result.status}")
    print(f"Tokens: {result.prompt_tokens + result.completion_tokens}")
    
    results.append({
        'question': query['question'],
        'expected': query['expected_sql'],
        'generated': result.sql,
        'status': result.status,
        'tokens': result.prompt_tokens + result.completion_tokens,
        'latency_ms': result.latency_ms
    })
    
    time.sleep(1)  # Rate limiting

# Analyze results
results_df = pd.DataFrame(results)
print("\n" + "=" * 80)
print("Summary Statistics:")
print(f"Success rate: {(results_df['status'] == 'success').mean():.1%}")
print(f"Avg tokens: {results_df['tokens'].mean():.0f}")
print(f"Avg latency: {results_df['latency_ms'].mean():.0f}ms")
"""
print("Live testing disabled. Uncomment the code above and set OPENAI_API_KEY to test.")

## 7. Retry Prompt Analysis

In [ ]:
# Examine retry prompt template
error_message = "Table 'customer' not found"
original_sql = "SELECT * FROM customer WHERE city = 'New York'"
question = "Show me all customers from New York"

retry_prompt = prompt_templates.build_retry_prompt(
    original_question=question,
    failed_sql=original_sql,
    error_message=error_message,
    schema=schema_manager.format_schema(format_type='create_table')
)

print("Retry Prompt:")
print("=" * 80)
print(retry_prompt)
print("=" * 80)
print(f"\nLength: {len(retry_prompt)} characters")
print(f"Estimated tokens: {len(retry_prompt) // 4}")

## 8. Key Findings and Recommendations

### Token Usage vs. Accuracy Trade-offs:

1. **Schema Format**:
   - `compact`: Lowest tokens (~100-200), good for simple queries
   - `create_table`: Medium tokens (~300-500), best balance
   - `json`: Similar to CREATE TABLE, more verbose

2. **Sample Rows**:
   - Add ~50-100 tokens per table
   - Helpful for understanding data format and examples
   - Most valuable for complex queries with specific data patterns

3. **Few-Shot Examples**:
   - 0 examples: ~200 total tokens (minimal context)
   - 3 examples: ~500 total tokens (recommended baseline)
   - 5 examples: ~800 total tokens (comprehensive)

### Recommendations:

1. **For Simple Queries**: Use compact schema, no samples, 1-2 examples
2. **For Medium Queries**: Use CREATE TABLE schema, no samples, 3 examples
3. **For Complex Queries**: Use CREATE TABLE schema, include samples, 5 examples
4. **Dynamic Selection**: Identify query complexity first, then adjust prompt accordingly

### Next Steps:
- Implement adaptive prompt selection based on query complexity
- A/B test different configurations on full test set
- Measure accuracy vs. cost trade-offs